# Column Functions y porque evitar usar UDFs en Apache Spark

Uno de los componentes que más me sorprendió en Apache Spark es que permitiera extender el vocabulario de SQL fuera de los límites de DSL con la ayuda de Column Functions o User-defined functions - UDFS, incluso incrustando funciones de negocio escritas en diferente lenguaje Scala, Java o Python.

## Contenido:
* [Prerequisitos](#head1)
* [Column Functions y User-defined functions - UDF en Apache Spark](#head2)
* [Consideraciones de rendimiento y orden de evaluación UDFs](#head3)
* [Column Functions for all!](#head4)
* [Reemplazando por for Column function](#head1)
* [Conclusiones](#head1)

## Prerequisitos<a class="anchor" id="head1"></a>

Unicamente necesitaremos las siguientes importaciones en nuestro notebook:

In [3]:
import org.apache.spark.sql.functions.{col, udf}
import org.apache.spark.sql.Column

import org.apache.spark.sql.functions.{col, udf}
import org.apache.spark.sql.Column


## Column Functions y User-defined functions - UDF en Apache Spark<a class="anchor" id="head2"></a>

Los Column Functions son funciones que reciben como parametro una(s) columna(s) y son capaces de retornar una(s) columna(s), se encuentran en el namespace <i style="color:blue;">org.apache.spark.sql.functions</i> (<a href="https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/functions.html">Java</a> <a href="https://spark.apache.org/docs/2.4.5/api/scala/index.html#org.apache.spark.sql.functions">Scala</a>). Al ser funciones nativas, pasan por el optimizador de consultas Catalyst. Escribimos la función <i style="color:blue;">square</i> una Column Functions que calcula el cuadrado de una columna:

In [8]:
def square(col:Column):Column = col * col
spark.range(1, 10).select(square(col("id"))).show

+---------+
|(id * id)|
+---------+
|        1|
|        4|
|        9|
|       16|
|       25|
|       36|
|       49|
|       64|
|       81|
+---------+



square: (col: org.apache.spark.sql.Column)org.apache.spark.sql.Column


User-defined functions o UDF es otra forma de crear funciones que extienden la funcionalidad de SQL, permitiendo construir complejas logicas de negocio y utilizarlas como si fueran funciones nativas de SQL y no relacionadas a tipos de datos asociados a Datasets. Los UDFs requieren ser registradas en Spark y estarán listas para su uso como funciones nativa de SQL. Spark serializará las funciones y las enviará a todos los procesos ejecutores en los worker para su ejecución. Reescribamos nuestra función <i style="color:blue;">square</i> como UDF:

In [9]:
def square(s: Long) = s * s
val squareUDF = udf(square(_:Long):Long)
spark.range(1, 10).select(squareUDF(col("id"))).show

+-------+
|UDF(id)|
+-------+
|      1|
|      4|
|      9|
|     16|
|     25|
|     36|
|     49|
|     64|
|     81|
+-------+



square: (s: Long)Long
squareUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,LongType,Some(List(LongType)))


Su versión Inline más compacta utilizada con Spark SQL

In [10]:
spark.udf.register("squareUDFInline", (s: Long) => s * s)
spark.range(1, 10).createTempView("square")
spark.sql("SELECT squareUDFInline(id) from square").show

+-----------------------+
|UDF:squareUDFInline(id)|
+-----------------------+
|                      1|
|                      4|
|                      9|
|                     16|
|                     25|
|                     36|
|                     49|
|                     64|
|                     81|
+-----------------------+



## Consideraciones de rendimiento y orden de evaluación UDFs<a class="anchor" id="head3"></a>

Existe una diferencia clave asociada con el lenguaje con el cual se escribio la UDF: si es Java o Scala,  correrán sobre las JVM en las maquinas esclavas, sin embargo si la función fué escrita en Python, Spark iniciara el proceso de Python en el worker, serializara la data a un formato aceptado por Python, ejecutara la función registro a registro en el proceso de Python y finalmente retornará los resultados a la JVM.

Estas diferencias en la forma de ejecución traen implicaciones a nivel de rendimiento(<a href="https://medium.com/@QuantumBlack/spark-udf-deep-insights-in-performance-f0a95a4d8c62">Spark UDF — Deep insights in performance</a>) ofreciendo evidencia de mejores resultados las UDFs escritas nativamente en Scala.

<img class="nh sg s t u he ai hn" srcset="https://miro.medium.com/max/552/1*FFi8Yk6mwSc6AvI-avWcYw.png 276w, https://miro.medium.com/max/1104/1*FFi8Yk6mwSc6AvI-avWcYw.png 552w, https://miro.medium.com/max/1280/1*FFi8Yk6mwSc6AvI-avWcYw.png 640w, https://miro.medium.com/max/1400/1*FFi8Yk6mwSc6AvI-avWcYw.png 700w" sizes="700px" role="presentation" src="https://miro.medium.com/max/1800/1*FFi8Yk6mwSc6AvI-avWcYw.png" width="1200" height="250">

Por que deberia evitar esta maravillosa funcionalidad aún siendo escrita en Scala? Los UDFs no son optimizados por el optimizador de consultas Catalys (<a href="https://blog.cloudera.com/working-with-udfs-in-apache-spark/">Working with UDFs in Apache Spark</a>) y las funciones nativas de SQL a menudo tendran un mejor rendimiento y deberían ser el primer enfoque considerado siempre que se pueda evitar la introducción de un UDF.

Adicional se debe tener encuenta que las operaciones de corto circuito
<img src="src/EvaluationOrderUDF.png">

## Column Functions for all!<a class="anchor" id="head4"></a>

En su mayoria de veces las Column Functions permitaran reescribir el código de una UDF existente a una versión nativa, los siguientes ejemplos nos podran mostrar su potencial:

### Column Functions con condicionales

In [16]:
def square(col:Column):Column = {
    when(col.leq(lit(5)), lit("less or equal to 5")).otherwise(lit("greater than 5")).as("comparative")
}
spark.range(1, 10).select(col("id"),square(col("id"))).show

+---+------------------+
| id|       comparative|
+---+------------------+
|  1|less or equal to 5|
|  2|less or equal to 5|
|  3|less or equal to 5|
|  4|less or equal to 5|
|  5|less or equal to 5|
|  6|    greater than 5|
|  7|    greater than 5|
|  8|    greater than 5|
|  9|    greater than 5|
+---+------------------+



square: (col: org.apache.spark.sql.Column)org.apache.spark.sql.Column


In [1]:
Antes de evitarlos deberíamos saber ¿que son los UDFs?, ¿cómo ayudan extendiendo las capacidades de SQL?, consideraciones respecto a el rendimiento, por que debería evitar esta maravillosa funcionalidad, ¡bienvenido Column Functions!

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-OU1IL3S:4040
SparkContext available as 'sc' (version = 2.4.0, master = local[*], app id = local-1587550523633)
SparkSession available as 'spark'


<console>: 24: error: not found: type Column

In [ ]:
Ya que no deberian ser nuestra primera elección las UDFs en Apache Spark, como podemos reemplazarlas?